# Setup

In [ ]:
#only run this if the next cell fails with "module not found" on skfeature
!pip install git+https://github.com/jundongl/scikit-feature.git

## Imports

In [ ]:
#basics
from math import floor, sqrt, pi
from random import sample
import time
import timeit
#helpful progress bar
from tqdm.notebook import tqdm, trange

#file access
import os
import glob
from google.colab import drive

#data processing
import numpy as np
import pandas as pd
#libraries for image trimming and resizing
from PIL import Image, ImageDraw, ImageFilter

#matplot
%matplotlib inline
import matplotlib.pyplot as plot
plot.rcParams["figure.figsize"] = (20, 10) # (w, h)

#scikit
#gotta love Python and the hard work everybody else put into this stuff
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, make_scorer
from skimage import io, exposure
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.filters import frangi, meijering, prewitt, gabor
from skimage.feature import hog
from skfeature.function.similarity_based.fisher_score import fisher_score, feature_ranking as rank_fisher
from skfeature.function.statistical_based.gini_index import gini_index, feature_ranking as rank_gini
from skfeature.utility.mutual_information import conditional_entropy
#torch
import torch
import torch.nn as nn
#tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

#Done
print("Modules loaded")

## File Loading

In [ ]:
#Access to Google Drive content
drive.mount('/content/drive')

#basic directory information
basePath = '/content/drive/My Drive/CSCE633HW5/'
print(os.listdir(basePath))
#directory structure for the project
train_dir_raw = basePath + 'train/'
test_dir_raw = basePath + 'test/'
train_dir = basePath + 'train_PREPROCESSED/' #destination of where images will be saved
test_dir = basePath + 'test_PREPROCESSED/' #destination of where images will be saved
train_dir_cnn = basePath + 'train_CNN/'
test_dir_cnn = basePath + 'test_CNN/'
#data files
train_data = pd.read_csv(basePath + 'train.csv')
test_data = pd.read_csv(basePath + 'test.csv')

#file listing and  loading
def test_extension(f):
  extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
  ftitle, fext = os.path.splitext(os.path.basename(f))
  for ext in extensions:
    if(fext == ext):
      return True
  return False
#
def normalize(img):
  return (img - img.mean()) / img.std()
#opens the image file as an skimage
def get_image(directory, f):
  return (img_as_float(io.imread(os.path.join(directory, f))))
#load the images referenced in the given data file
#this ensures they are in the same order
def get_imgs(dataframe,directory):
  imgs = []
  for i in trange(len(dataframe)):
      ig = get_image(directory, dataframe.iloc[i]['filename']) #match the order of the images in the dataframe and train_dir 
      imgs.append(ig)                                           #add all the images in a big array 
  return np.array(imgs,dtype='float32')
#get the labels (outcome) as an array 
def get_labels(dataframe):
  labels = dataframe['covid(label)']
  return labels.to_numpy()

# Preprocessing

In [ ]:
def crop_center(pil_img, crop_width, crop_height): #manual definition of crop_center
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

#Function for getting largest square from center (shaves off excess height, maintain current width)
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

#function that takes in source folder of images, destination folder to save, and image width
#The for loop reads each image file, converts to RGB, resizes using LANCZOS filter and saves it to dst_dir
def preprocess(src_dir,dst_dir,imageWidth):
  files = [f for f in os.listdir(src_dir) if test_extension(f)]
  print("Processing", src_dir, "->", dst_dir, f"({imageWidth}x{imageWidth})")
  for f in tqdm(files):
    im = Image.open(os.path.join(src_dir, f))
    im = im.convert("RGB") # important, not all are JPEG. Converts from RGBA to JPEG compatible
    im_thumb = crop_max_square(im).resize((imageWidth, imageWidth), Image.LANCZOS)
    im_thumb.save(os.path.join(dst_dir, f), quality=95)
  print("Processed", len(files), "files")


In [ ]:
preprocess(train_dir_raw, train_dir, 200)
preprocess(test_dir_raw, test_dir, 200)
preprocess(train_dir_raw, train_dir_cnn, 100)
preprocess(test_dir_raw, test_dir_cnn, 100)

# Load the training files

In [ ]:
#load the datasets
X_train_img = get_imgs(train_data, train_dir)
#X_train = features(X_img_train)
X_train_cnn = get_imgs(train_data, train_dir_cnn)
y_train = get_labels(train_data)

# Feature Extraction and Visualization

In [ ]:
#prepares n subplot boxes
def npaxes(n):
    #
    r = floor(sqrt(n))
    while(n % r != 0):
        r -= 1
    c = int(n / r)
    #
    fig, ax = plot.subplots(r, c)
    r_ = 0
    c_ = 0

    axes = []

    for _ in range(n):
        if(c == 1 or r == 1):
          axes.append(ax[max(r_, c_)])
        else:
          axes.append(ax[r_, c_])
        #
        c_ += 1
        if(c_ >= c):
            c_ = 0
            r_ += 1

    #plot.tight_layout()
    #plot.show()

    return axes

#I wanted to use meijering neuriteness / frangi edge filters
#but they didn't really work - prewitt filter seemed to extract edges much better

#extracts the following features:
#-two gabor filters
#-prewitt edge filter
#-HOG
#also generates visualization data for each
#so it returns a pair, first element = feature vector, second element = visualization stuff
def extract_features(img):
    img = rgb2gray(img)
    g1 = gabor(img, 1, theta = 0)
    g2 = gabor(img, 1, theta = pi / 2)
    p = prewitt(img)
    (hog_fd, hog_img) = hog(img, visualize=True)
    hog_img = exposure.rescale_intensity(hog_img, in_range=(0, 10))
    #
    #features = [np.histogram(feat, bins='sqrt') for feat in [g1[0], g2[0], p]] + [hog_fd]
    features = np.concatenate((g1[0].flatten(), g2[0].flatten(), p.flatten(), hog_fd))
    visualizations = [img, g1[0], g2[0], p, hog_img]
    labels = ["Src", "Gabor1", "Gabor2", "Prewitt", "HOG"]
    return (features, (visualizations, labels))

def display_features(vl):
    (vis, labels) = vl
    n = len(vis)
    axes = npaxes(n * 2)
    for i in range(n):
        f = vis[i]
        axes[i].imshow(f, cmap="gray")
        axes[i].set_xlabel(labels[i])
        axes[i + n].hist(f.flatten(), bins="sqrt")
        axes[i + n].set_xlabel(labels[i])

def features(imgs):
    feats = []
    vis = []
    for img in tqdm(imgs, desc="Extracting features"):
        (feat, v) = extract_features(img)
        feats.append(feat)
        vis.append(v)
    return (np.stack(feats), np.stack(vis)) #.array and .stack should do the same thing here, I believe

In [ ]:
(X_train, X_train_vis) = features(X_train_img)

#visualize one random point
display_features(X_train_vis[sample(range(X_train_vis.shape[0]), 1)[0]])

## Feature Scoring

In [ ]:
#unfortunately, there isn't any way to add a progress bar to the skfeature methods.
# :(

#conditional entropy
def all_entropies(X, y):
  feats = X.shape[1] #number of features
  entropies = [0] * feats
  for f in trange(feats):
    Xs = X[:,f]
    entropies[f] = conditional_entropy(Xs, y)
  return entropies

#returns the indices of entries in ascending order
def sort_ranks(xs):
  n = len(xs)
  #associate each with its index
  ids = zip(xs, range(n))
  #sort by value and then retrieve index
  return [x[1] for x in sorted(ids, key = lambda t: t[0])]

#entropy_scores = all_entropies(X_train, y_train)
entropy_rank = sort_ranks(entropy_scores)
entropy_n = 10
print("Top", entropy_n, "features (entropy):", entropy_rank[:entropy_n])

#fisher score
#fisher_scores = fisher_score(X_train, y_train) #score for each feature
#fisher_rank = rank_fisher(fisher_scores) #feature indices sorted by fisher score
fisher_n = 10
print("Top", fisher_n, "features (fisher):", fisher_rank[:fisher_n])

#gini index (takes a long time to calculate)
#gini_indices = gini_index(X_train, y_train)
#gini_rank = rank_gini(gini_indices)
gini_n = 10
print("Top", gini_n, "features (gini):", gini_rank[:gini_n])

In [ ]:
#plot histograms of the distribution of each of the most important features

def display_best_features(X, features):
    n = len(features)
    axes = npaxes(n)
    for i in range(n):
        f = features[i]
        data = X[:,f]
        axes[i].set_xlabel("Feature " + str(f))
        axes[i].hist(data, bins="sqrt")

#remove duplicates and sort (makes it back into a list)
def get_best_features():
  a = entropy_rank[:entropy_n]
  b = fisher_rank[:fisher_n].tolist()
  c = gini_rank[:gini_n].tolist()
  return sorted(set(a + b + c))

best_features = get_best_features()
print("Best features:", best_features)
display_best_features(X_train, best_features)

## Feature Selection

In [ ]:
#Filter Method
#use SelectKbest from sklearn that takes the matrix features, the labels
#and number of features to select (k) then performs statistical tests to
#select k number of features 
#the number of features targeted is 50

skb = SelectKBest(chi2, k=50)
X_filter = skb.fit_transform(X_train, y_train)
print("Filter-method feature indices:", skb.get_support(indices = True)) #this prints the indices of the selected features

#The wrapper method used is the Recursive Feature Elimination (RFE) 
#it works by recursively removing features and building a model on the remaining
#this method would go over all the features (162,000) and as a result takes a lot of time to run
#A way to lower the computational time for the wrapper method is to use 
#the filter method initially and apply it to get 2000 feature out of the 162,000
#then the REF is applied to the 1000 features to select 50 features


#Filter method 
skb2 = SelectKBest(chi2, k=1000)
X_filter2 = skb2.fit_transform(X_train, y_train)
#print(skb2.get_support(indices = True))

#Wrapper method 

model = LogisticRegression(solver='lbfgs', max_iter=10000)
rfe = RFE(model, 50)
X_wrapper2 = rfe.fit_transform(X_filter2, y_train)
print("Wrapper-method feature indices:", skb2.get_support(indices = True)[rfe.get_support(indices = True)]) #this prints the indicies of the selected features

In [ ]:
#used to test each of the features
def cross_val(X, y):
  random_index = np.random.permutation(len(X))
  scores = []
  for i in range(0,5):
      #initilize the training/testing lists
      Xs = []
      Xt = []
      ys = []
      yt = []
      #split into 5 folds randomly
      for j in range(0,len(X)): 
          if (j%5 ==i):
              Xs.append(X[random_index[j]])
              ys.append(y[random_index[j]])
          else: 
              Xt.append(X[random_index[j]])
              yt.append(y[random_index[j]])
      #convert list to array
      Xs = np.asarray(Xs)
      Xt = np.asarray(Xt)
      ys = np.asarray(ys)
      yt = np.asarray(yt)
      
      #apply logistic regression to the testing data
      model = LogisticRegression(solver='saga', max_iter=10000)
      #C = Inverse of regularization strength; must be a positive float --> smaller values specify stronger regularization.
      model.fit(Xt, yt)
      pred = model.predict(Xs)
      score = model.score(Xs, ys)
      scores += [score]
      print((round(score,3)))

      # plot the performance 
      plot.plot(range(1, len(scores)+1), scores)
      plot.yticks([0, 0.5, 1])
      plot.xticks([0, 1, 2, 3, 4, 5])
      plot.xlabel("Subset of features", fontsize = 25)
      plot.ylabel("Cross validation score ", fontsize =25)
      # plot.show()
  print("Mean score:", sum(scores) / len(scores))

In [ ]:
#apply cross validation to filter method features
cross_val(X_filter, y_train)

In [ ]:
#apply the cross validation to the wrapper method features
cross_val(X_wrapper2, y_train)

## AdaBoost

In [ ]:
def adaboost(T, cv_count = 5):
  classifier = AdaBoostClassifier(LogisticRegression(max_iter=10000), n_estimators=T)
  scores = cross_validate(classifier, X_train, y_train, cv=cv_count, scoring = make_scorer(accuracy_score))["test_score"]
  print("AdaBoost(", T, ") test scores:", scores)
  print("Average:", sum(scores)/len(scores))

In [ ]:
adaboost(1)
adaboost(10)
adaboost(100)
adaboost(1000)

# Improving Performance

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')


#Reference link for cross-validation with CNN: https://medium.com/@navmcgill/k-fold-cross-validation-in-keras-convolutional-neural-networks-835bed559d04
def optimize_cnn(hyperparameter):
  
  layers = [Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation'], input_shape=(100,100,3)), 
            Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation']), 
            MaxPooling2D(pool_size=(2,2),padding='same'), Dropout(hyperparameter['dropout_prob']),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation']),
            Conv2D(64, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation']), 
            MaxPooling2D(pool_size=(2,2),padding='same'), Dropout(hyperparameter['dropout_prob']),
            Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation']),
            Conv2D(32, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], activation=hyperparameter['activation']),
            MaxPooling2D(pool_size=(2,2),padding='same'), Dropout(hyperparameter['dropout_prob']),
            Flatten(),
            Dense(512,activation='relu'), #dense layer 1
            Dense(256,activation='relu'), #dense layer 2
            Dense(2,activation='softmax'), #dense layer 3
            ] 

  # Define model using hyperparameters 
  used_layers = [layers[i] for i in hyperparameter['layer_ids']]+layers[12:]
  cnn_model = Sequential(used_layers)
  #print(cnn_model.summary())
  cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  v_scores = []
  t_scores = []
  start_time = time.time()
  for i in range(5):
    tqdm_callback = tfa.callbacks.TQDMProgressBar()
    X_train_fold, X_val_fold, y_train_fold, y_val_fold = train_test_split(X_train_cnn, y_train, test_size=0.20, random_state = np.random.randint(1,1000, 1)[0])
    results = cnn_model.fit(X_train_fold, to_categorical(y_train_fold), validation_data=(X_val_fold, to_categorical(y_val_fold)), epochs=30, batch_size=52, verbose=0)
    t_performance = results.history['accuracy'][-1]
    _, v_performance = cnn_model.evaluate(X_val_fold, to_categorical(y_val_fold), verbose = 0)
    v_scores.append(v_performance)
    t_scores.append(t_performance)
  end_time = time.time()
  print("Training accuracy: {}\nTraining time: {}".format(sum(t_scores)/len(t_scores), end_time-start_time))

  # Evaluate accuracy on validation data
  #performance = cnn_model.evaluate(fnn_val_proc, to_categorical(val_labels), verbose=0)
  averagePerformance = sum(v_scores)/len(v_scores)
  print("Hyperparameters: ", hyperparameter, "Validation Accuracy: ", averagePerformance)
  print("----------------------------------------------------")
  # We want to minimize loss i.e. negative of accuracy
  return({"status": STATUS_OK, "loss": -1*averagePerformance, "model":cnn_model,"params":hyperparameter,"acc":averagePerformance})

# Define search space for hyper-parameters
space = {
    'layer_ids': hp.choice('layer_ids',[[0,2,3,4,6,7],[0,1,2,3],[0,2,3,8,10,11],[0,2,3]]),
    'conv_kernel_size': hp.choice('conv_kernel_size',[1,3,5]),
    'conv_stride': hp.choice('conv_stride',[1,2]),
    'activation': hp.choice('activation', ['relu','selu','tanh']),
    'dropout_prob': hp.uniform('dropout_prob',0,0.35),
}

trials = Trials()
with tf.device('/device:GPU:0'):
  # Find the best hyperparameters
  best = fmin(
          optimize_cnn,
          space,
          algo=tpe.suggest,
          trials=trials,
          max_evals=10,
          verbose=0,
          show_progressbar=False
      )
  
  test_result = trials.results[np.argmin([r['loss'] for r in trials.results])] #best trial info
  best_model = test_result['model']
  best_params = test_result['params']
  best_acc = test_result['acc']

  print("\n==================================\n")
  print("Best Hyperparameters:", best_params) #the output here doesn't makes sense because it prints indices into the hp.choice objects
  print("Accuracy:", best_acc)

  # You can retrain the final model with optimal hyperparameters on train+validation data

  # Or you can use the model returned directly
  # Find trial which has minimum loss value and use that model to perform evaluation on the test data
  #test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

  #for bonus question if we have time :)
  #performance = test_model.evaluate(fnn_test_proc, to_categorical(test_labels))

  #print("==================================")
  #print("Test Accuracy: ", performance[1])